In [1]:
!pip install opencv-python

In [3]:
!pip install matplotlib

In [5]:
!pip install numpy

In [3]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import math

def compute_major_axis(contour):
    max_dist = 0
    pt1, pt2 = None, None
    for i in range(len(contour)):
        for j in range(i+1, len(contour)):
            dist = np.linalg.norm(contour[i] - contour[j])
            if dist > max_dist:
                line = np.linspace(contour[i], contour[j], 1000)
                inside = True
                for point in line:
                    if cv2.pointPolygonTest(contour, (point[0][0], point[0][1]), False) < 0:
                        inside = False
                        break
                if inside:
                    max_dist = dist
                    pt1 = tuple(contour[i][0])
                    pt2 = tuple(contour[j][0])
    return max_dist, pt1, pt2

def add_padding(img, border_size=50):
    """Add a white padding around the image."""
    return cv2.copyMakeBorder(img, top=border_size, bottom=border_size, left=border_size, right=border_size, borderType=cv2.BORDER_CONSTANT, value=[255, 255, 255])

input_folder = '/content/drive/MyDrive/Cognida Coding Challenge/Mineral Processing Technology - Image Analytics/input'
output_folder = '/content/drive/MyDrive/Cognida Coding Challenge/Mineral Processing Technology - Image Analytics/output'

visualized_outputs = []

for filename in os.listdir(input_folder):
    filepath = os.path.join(input_folder, filename)
    original_image = cv2.imread(filepath)
    gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Add padding
        padded_image = add_padding(original_image.copy())

        # Adjust the contour points based on the padding
        contour += 50  # 50 is the border_size as set in add_padding function

        # 1. Smallest Enclosing Circle
        image = padded_image.copy()
        (x, y), radius = cv2.minEnclosingCircle(contour)
        center = (int(x), int(y))
        cv2.circle(image, center, int(radius), (57, 0, 199), 2)
        output_filepath = os.path.join(output_folder, f"encircle_{filename}")
        cv2.imwrite(output_filepath, image)

        # 2. Total Surface Area
        image = padded_image.copy()
        area = cv2.contourArea(contour)
        text_position = (image.shape[1] - 250, 50)  # Display the text on the upper right side
        cv2.putText(image, f"Area={int(area)} pixels", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.75, (57, 0, 199), 2)
        output_filepath = os.path.join(output_folder, f"area_{filename}")
        cv2.imwrite(output_filepath, image)

        # 3. Major Axis
        image = padded_image.copy()
        major_axis, pt1, pt2 = compute_major_axis(contour)
        cv2.line(image, pt1, pt2, (0, 0, 255), 2)
        cv2.putText(image, f"Major axis={int(major_axis)} pixels", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.75, (57, 0, 199), 2)
        output_filepath = os.path.join(output_folder, f"major_axis_{filename}")
        cv2.imwrite(output_filepath, image)

        # 4. Perimeter
        image = padded_image.copy()
        perimeter = cv2.arcLength(contour, True)
        cv2.putText(image, f"Perimeter={int(perimeter)} pixels", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.75, (57, 0, 199), 2)
        output_filepath = os.path.join(output_folder, f"perimeter_{filename}")
        cv2.imwrite(output_filepath, image)

        # 5. Centroid
        image = padded_image.copy()
        M = cv2.moments(contour)
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        cv2.circle(image, (cx, cy), 5, (36, 184, 233), -1)
        cv2.putText(image, f"Centroid=({cx},{cy})", (center[0]-55, center[1]+60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (57, 0, 199), 2)
        output_filepath = os.path.join(output_folder, f"centroid_{filename}")
        cv2.imwrite(output_filepath, image)

plt.figure(figsize=(15, 10))
columns = 5

for i, image in enumerate(visualized_outputs, 1):
    plt.subplot(math.ceil(len(visualized_outputs) / columns), columns, i)
    plt.imshow(image)
    plt.axis('off')

plt.show()


<Figure size 1500x1000 with 0 Axes>